In [1]:
import pandas as pd
pd.options.display.max_columns = None

# Load dataframe with first column as the index
df = pd.read_csv('../rec_sys_alumnos.csv', index_col=0, dtype=str)

# Trim spaces at the beginning and at the end of each record
df = df.apply(lambda x: x.str.strip())

# Set 'NA' strings as nan value
df = df.replace('NA', float('nan'))

columns = df.columns.tolist()

# Manual classification of columns by their type (products are already binary)
asint = [0, 4, 7, 8, 9, 18]
products = list(range(23, 48))
asfloat = [20, 22]
asdate = [1, 5, 10]
ascategory = [2, 6, 11, 12, 15, 21]
asbinary = [3, 13, 14, 16, 17, 19]

# Transform some columns to binary
from numpy import mod
for i in asbinary:
    options = pd.Series(df[columns[i]].unique()).sort_values().values
    binary = [0, 1]
    for j in [-2, -3]:
        df[columns[i]] = df[columns[i]].replace(options[mod(j, len(options))], binary[mod(j, len(binary)+1)])

# Convert numeric columns to float (including binary)
for i in asfloat + asint + products + asbinary:
    df[columns[i]] = df[columns[i]].astype('float64')

# Try to convert integer columns to int (not possible if columns contains nan, since this value is float)
for i in asint + products + asbinary:
    try:
        df[columns[i]] = df[columns[i]].astype('int64')
    except ValueError:
        continue

# Convert date columns to datetime
for i in asdate:
    df[columns[i]] = pd.to_datetime(df[columns[i]])

# Convert some columns to categorical
for i in ascategory:
    df[columns[i]] = df[columns[i]].astype('category')

df

,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,fec_ult_cli_1t,xti_rel_1mes,tip_rel_1mes,indresi,indext,des_canal,xti_extra,tip_dom,cod_provincia,xti_actividad_cliente,imp_renta,id_segmento,mean_engagement,ind_prod1,ind_prod2,ind_prod3,ind_prod4,ind_prod5,ind_prod6,ind_prod7,ind_prod8,ind_prod9,ind_prod10,ind_prod11,ind_prod12,ind_prod13,ind_prod14,ind_prod15,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,178103,2015-01-28,ES,0.0,35.0,2015-01-12,N,0.0,6.0,1.0,NaT,1.0,A,1.0,0.0,KHL,0.0,1,29.0,1.0,87218.10,02 - PARTICULARES,6.134600,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
1,503082,2015-01-28,ES,1.0,27.0,2012-08-10,N,0.0,35.0,1.0,NaT,1,I,1.0,0.0,KHE,0.0,1,40.0,0.0,70777.59,03 - UNIVERSITARIO,5.414673,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
2,502996,2015-01-28,ES,1.0,37.0,2012-08-10,N,0.0,35.0,1.0,NaT,1.0,A,1.0,0.0,KFC,0.0,1,8.0,0.0,104035.08,02 - PARTICULARES,6.301283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
3,503053,2015-01-28,ES,0.0,23.0,2012-08-10,N,0.0,35.0,1.0,NaT,1,A,1.0,0.0,KHE,0.0,1,39.0,1.0,136930.23,03 - UNIVERSITARIO,5.014352,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1,0
4,503031,2015-01-28,ES,0.0,44.0,2012-08-10,N,0.0,35.0,1.0,NaT,1.0,I,1.0,0.0,KAT,0.0,1,28.0,0.0,110245.53,02 - PARTICULARES,6.851770,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641702,256354,2016-04-28,ES,1.0,25.0,2014-08-18,N,0.0,25.0,1.0,NaT,1,A,1.0,0.0,RED,0.0,1,30.0,1.0,81987.18,02 - PARTICULARES,5.218520,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1,0
641703,256273,2016-04-28,ES,0.0,65.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KAT,0.0,1,41.0,0.0,131135.70,02 - PARTICULARES,8.280778,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
641704,256253,2016-04-28,ES,1.0,42.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KFC,0.0,1,8.0,0.0,74745.30,02 - PARTICULARES,6.699261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
641705,256296,2016-04-28,ES,0.0,42.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KFC,0.0,1,28.0,0.0,215942.10,02 - PARTICULARES,6.699261,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0


In [ ]:
# HERE GOES DATA IMPUTATION OR NA CLEANING

In [2]:
meses = pd.Series(df['mes'].unique()).sort_values().values

# List of dataframes. Each dataframe contains the records corresponding to one month
df_xmes = [df[df['mes'] == mes] for mes in meses]

# Create a new DataFrame with the same features but the products and the date
df_status = df[[columns[i] for i in range(0, 23)]].copy()
# List of dataframes. Each dataframe contains the records corresponding to in-between months
df_status_xmes = [df_status[df_status['mes'] == mes].drop('mes', axis=1) for mes in meses[1:]]

In [3]:
# For each month calculate the product status with respect to the previous month, and store it in "df_status_xmes".
# After each calculation, append the DataFrame to new DataFrame "df_status"
del(df_status)
df_status = pd.DataFrame(columns=columns)

for i in range(len(df_xmes) - 1):
    product_status = df_xmes[i+1].merge(df_xmes[i][[columns[p0] for p0 in [0]+products]], on='cod_persona', how='left')
    for p in products:
        df_status_xmes[i][columns[p]] = product_status[columns[p]+'_x'].fillna(0).values - product_status[columns[p]+'_y'].fillna(0).values
    df_status_xmes[i]['mes'] = str(meses[i])[:7] + ' \u2192 ' + str(meses[i+1])[:7]
    df_status = df_status.append(df_status_xmes[i])

# For some reason, integer columns are corrupted. Let's try to convert them into int again
for i in asint + products + asbinary:
    try:
        df_status[columns[i]] = df_status[columns[i]].astype('int64')
    except ValueError:
        continue

df_status

,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,fec_ult_cli_1t,xti_rel_1mes,tip_rel_1mes,indresi,indext,des_canal,xti_extra,tip_dom,cod_provincia,xti_actividad_cliente,imp_renta,id_segmento,mean_engagement,ind_prod1,ind_prod2,ind_prod3,ind_prod4,ind_prod5,ind_prod6,ind_prod7,ind_prod8,ind_prod9,ind_prod10,ind_prod11,ind_prod12,ind_prod13,ind_prod14,ind_prod15,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
31537,1008590,2015-01 → 2015-02,ES,1.0,45.0,2005-08-02,N,0.0,120.0,1.0,NaT,1.0,I,1.0,0.0,KAT,0.0,1,28.0,1.0,62736.12,02 - PARTICULARES,6.926724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31538,1008582,2015-01 → 2015-02,ES,0.0,65.0,2005-08-02,N,0.0,120.0,1.0,NaT,1.0,I,1.0,0.0,KBZ,0.0,1,28.0,0.0,NaN,02 - PARTICULARES,8.280778,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31539,1008547,2015-01 → 2015-02,ES,1.0,52.0,2005-08-02,N,0.0,120.0,1.0,NaT,1.0,A,1.0,0.0,KFC,0.0,1,39.0,1.0,64413.75,02 - PARTICULARES,7.429623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31540,1008663,2015-01 → 2015-02,ES,1.0,48.0,2005-08-24,N,0.0,119.0,1.0,NaT,1.0,A,1.0,0.0,KFA,0.0,1,28.0,1.0,1242325.23,01 - TOP,7.146723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31542,1008466,2015-01 → 2015-02,ES,1.0,49.0,2005-08-03,N,0.0,119.0,1.0,NaT,1,I,1.0,0.0,KFC,0.0,1,43.0,0.0,66302.13,02 - PARTICULARES,7.218520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641702,256354,2016-03 → 2016-04,ES,1.0,25.0,2014-08-18,N,0.0,25.0,1.0,NaT,1,A,1.0,0.0,RED,0.0,1,30.0,1.0,81987.18,02 - PARTICULARES,5.218520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
641703,256273,2016-03 → 2016-04,ES,0.0,65.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KAT,0.0,1,41.0,0.0,131135.70,02 - PARTICULARES,8.280778,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
641704,256253,2016-03 → 2016-04,ES,1.0,42.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KFC,0.0,1,8.0,0.0,74745.30,02 - PARTICULARES,6.699261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
641705,256296,2016-03 → 2016-04,ES,0.0,42.0,2014-08-18,N,0.0,20.0,1.0,NaT,1,I,1.0,0.0,KFC,0.0,1,28.0,0.0,215942.10,02 - PARTICULARES,6.699261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now in ```ind_prodX``` I have ```0``` if the product was kept or never contracted in the corresponding period, ```-1``` if the product was abandoned and ```1``` if the product was contracted.